In [7]:
from collections import defaultdict
from datetime import datetime
import json
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm

sys.path.append(os.path.abspath(".."))

from config import (
    RAW_DATA_PATH,
    FILTERED_DATA_PATH,
    TRAIN_DATA_PATH,
    VAL_DATA_PATH,
    TEST_DATA_PATH,
    METADATA_PATH,
    EMBEDDINGS_TITLE_PATH,
    EMBEDDINGS_DESCR_PATH,
    EPISODE_DESCRIPTION_PATH,
    RANDOM_STATE,
    N_RECOMMENDATIONS,
    N_EPOCHS,
    RECOMMENDATIONS_PATH,
    TRANSFORMED_DATA_PATH,
    SPLIT_DATE_TRAIN_VAL,
    SPLIT_DATE_VAL_TEST,
)
import utils.utils as utils

In [8]:
# importing raw data
raw_df = pd.read_parquet("../" + RAW_DATA_PATH)

# importing training data
train_df = pd.read_parquet("../" + TRAIN_DATA_PATH)

# importing validation data
val_df = pd.read_parquet("../" + VAL_DATA_PATH)

# importing test data
test_df = pd.read_parquet("../" + TEST_DATA_PATH)

# importing metadata
meta_df = pd.read_parquet("../" + METADATA_PATH)

# importing episode descriptions
descr_df = pd.read_parquet("../" + EPISODE_DESCRIPTION_PATH)

# importing the transformed data
transformed_df = pd.read_parquet("../" + TRANSFORMED_DATA_PATH)

# importing the filtered data
filtered_df = pd.read_parquet("../" + FILTERED_DATA_PATH)

# importing title embeddings
title_embeddings = pd.read_parquet("../" + EMBEDDINGS_TITLE_PATH)

# importing description embeddings
descr_embeddings = pd.read_parquet("../" + EMBEDDINGS_DESCR_PATH)

In [37]:
train_w_meta = train_df.merge(meta_df, how="left", on="prd_number")

In [46]:
show_counts = train_w_meta.groupby("series_title")["user_id"].nunique()
show_counts.sort_values(ascending=False)[:10].index.tolist()
# top_10_shows = show_counts.nlargest(10, "count")["series_title"].tolist()
# top_10_shows

['Genstart',
 'Tiden',
 'Stjerner og striber',
 'Djævlen i detaljen',
 'Lyssky',
 'Sara & Monopolet - podcast',
 'Tyran',
 'Ubegribeligt',
 'Bakspejl',
 'Udsyn']

In [42]:
show_counts = train_w_meta.groupby("series_title").size().reset_index(name="count")
top_10_shows = show_counts.nlargest(10, "count")["series_title"].tolist()
top_10_shows

['Genstart',
 'Tiden',
 'Djævlen i detaljen',
 'Stjerner og striber',
 'Radioavisen',
 'Sara & Monopolet - podcast',
 'Tyran',
 'Lyssky',
 'Ubegribeligt',
 'Bakspejl']

In [36]:
genstart_df = transformed_df[transformed_df["series_title"] == "Genstart"]
genstart_df["genre"].value_counts()

genre
Aktualitet    425109
-                922
Name: count, dtype: int64

In [30]:
print(len(train_df["user_id"].unique()))
print(len(val_df))
print(len(test_df))

44007
581287
556116


In [18]:
raw_df[raw_df["Content Type (revar3)"] == "vod"]["Genre"].value_counts()

Genre
Fakta og debat     5011755
Aktualitet         2947147
Livsstil           1925771
Unge               1725776
Nyheder            1503841
Kultur             1159418
Musik               969257
-                   885500
Historie            587171
Dokumentar          275269
Sport               167713
NN                   29736
Underholdning        25754
Kor og orkestre        100
Name: count, dtype: int64

In [86]:
import pandas as pd

# Load your data
df = pd.read_csv("../results/cf_experiments.csv", header=0)  # or manually create the DataFrame
print(df)
# Select only the independent variables
df.columns = df.columns.str.strip()

X = df[['n_components', 'damping', 'reg']]

# Compute pairwise Pearson correlations
correlation_matrix = X.corr()

print(correlation_matrix)


     n_components   damping    reg   ndcg@10
0              50         1  0.050  0.010545
1              50         1  0.100  0.002846
2              50         1  0.150  0.000797
3              50         5  0.050  0.017417
4              50         5  0.100  0.006394
..            ...       ...    ...       ...
103           800       100  0.003  0.068805
104           800       100  0.004  0.069434
105           800       120  0.002  0.067800
106           800       120  0.003  0.069293
107           800       120  0.004  0.069966

[108 rows x 4 columns]
              n_components   damping       reg
n_components      1.000000  0.930548 -0.614693
damping           0.930548  1.000000 -0.620835
reg              -0.614693 -0.620835  1.000000
